In [ ]:
install.packages(c("dplyr","TSstudio","forecast","tidyr","magrittr","arsenal"))
#install.packages("arsenal")

In [1]:
library(dplyr)
library(TSstudio)
library(forecast)
library(tidyr)
library(plotly)
library(ggplot2)
library(arsenal)
library(magrittr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 

Registered S3 methods overwritten by 'forecast':
  method             from    
  fitted.fracdiff    fracdiff
  residuals.fracdiff fracdiff

Loading required package: ggplot2


Attaching package: ‘plotly’


The following object is masked from ‘package:ggplot2’:

    last_plot


The following object is masked from ‘package:stats’:

    filter


The following object is masked from ‘package:graphics’:

    layout



Attaching package: ‘magrittr’


The following object is masked from ‘package:arsenal’:

    set_attr


The following object is masked from ‘package:tidyr’:

    extract




In [2]:
scenario1 <- read.csv("scenario1_output.csv")
scenario2 <- read.csv("scenario2_output.csv")
scenario1 <- scenario1[, colSums(scenario1 != 0) > 0]
scenario2 <- scenario2[, colSums(scenario2 != 0) > 0]

In [3]:
# Columns should match, if not one of the models is returning some error and we need to remove it for both scripts
# remove column by name for scenario1 and 2...

# check to see if columns match
summary(comparedf(scenario1,scenario2))



Table: Summary of data.frames

version   arg          ncol   nrow
--------  ----------  -----  -----
x         scenario1     309    480
y         scenario2     309   1584



Table: Summary of overall comparison

statistic                                                      value
------------------------------------------------------------  ------
Number of by-variables                                             0
Number of non-by variables in common                             309
Number of variables compared                                     308
Number of variables in x but not y                                 0
Number of variables in y but not x                                 0
Number of variables compared with some values unequal            307
Number of variables compared with all values equal                 1
Number of observations in common                                 480
Number of observations in x but not y                              0
Number of observations in y

In [4]:
str(scenario1)
str(scenario2)

'data.frame':	480 obs. of  309 variables:
 $ Date                                        : Factor w/ 48 levels "2019-01-01","2019-02-01",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ Scenario                                    : Factor w/ 1 level "Field Ops - Mass Markets - Test 7/7/2021": 1 1 1 1 1 1 1 1 1 1 ...
 $ Product                                     : Factor w/ 3 levels "EAST","NORTHERN",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Sub.Product                                 : Factor w/ 10 levels "AZ","FL_TX","GREAT PLAINS",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ Volumes                                     : int  52567 45137 50092 51663 46423 46858 56177 53765 43548 48903 ...
 $ c1_AAA_ets                                  : num  0 0 0 0 0 0 0 0 0 0 ...
 $ c1_AAN_ets                                  : num  0 0 0 0 0 0 0 0 0 0 ...
 $ c1_add_holtlineartrend                      : num  0 0 0 0 0 0 0 0 0 0 ...
 $ c1_add_holtlineartrenddamp                  : num  0 0 0 0 0 0 0 0 0 0 ...
 $ c1_ANA_ets                 

In [5]:
# This will filter models with names that match with regex pattern by least MAD based off test data
# I am showing the functionality here for regex pattern of c1_ 
#, you would have to change the regex pattern for c1_sarima to get the best sarima model 
# .. out of the x amount of sarima models, and do the same for both scenario dataframes
# after that, since we have one sarima column, we would need to rename for both scenario1 and 2 the column to have a common name
# Repeat for all other models

# Code works fine without this, and MAD is calculated within PowerBi anyway.. but it is here if needed.
# Make sure that it is calculating best MAD for all product/subproducts, might need to add code to keep that in mind

 scenario1 %>% 
   filter(if_all(c(Volumes, matches('^c\\d+_\\w+$')), ~ . != 0))  %>% 
   summarise(across(matches('^c\\d+_\\w+$'), ~ mean(abs(Volumes - .)))) %>% 
   pivot_longer(everything()) %>%
   filter(value != min(value)) %$% 
   select(scenario1, -all_of(name)) 

 scenario2 %>% 
    filter(if_all(c(Volumes, matches('^c\\d+_\\w+$')), ~ . != 0))  %>% 
    summarise(across(matches('^c\\d+_\\w+$'), ~ mean(abs(Volumes - .)))) %>% 
    pivot_longer(everything()) %>%
    filter(value != min(value)) %$% 
    select(scenario2, -all_of(name)) 

Date,Scenario,Product,Sub.Product,Volumes,c1_sarima1
2019-01-01,Field Ops - Mass Markets - Test 7/7/2021,EAST,FL_TX,52567,0
2019-02-01,Field Ops - Mass Markets - Test 7/7/2021,EAST,FL_TX,45137,0
2019-03-01,Field Ops - Mass Markets - Test 7/7/2021,EAST,FL_TX,50092,0
2019-04-01,Field Ops - Mass Markets - Test 7/7/2021,EAST,FL_TX,51663,0
2019-05-01,Field Ops - Mass Markets - Test 7/7/2021,EAST,FL_TX,46423,0
2019-06-01,Field Ops - Mass Markets - Test 7/7/2021,EAST,FL_TX,46858,0
2019-07-01,Field Ops - Mass Markets - Test 7/7/2021,EAST,FL_TX,56177,0
2019-08-01,Field Ops - Mass Markets - Test 7/7/2021,EAST,FL_TX,53765,0
2019-09-01,Field Ops - Mass Markets - Test 7/7/2021,EAST,FL_TX,43548,0
2019-10-01,Field Ops - Mass Markets - Test 7/7/2021,EAST,FL_TX,48903,0


Date,Scenario,Product,Sub.Product,Volumes,c1_sarima1
2019-01-01,Service Delivery - RBO + SDP + EASE - Te,CCE,Change,58.47826,0
2019-02-01,Service Delivery - RBO + SDP + EASE - Te,CCE,Change,1.00000,0
2019-03-01,Service Delivery - RBO + SDP + EASE - Te,CCE,Change,131.00000,0
2019-04-01,Service Delivery - RBO + SDP + EASE - Te,CCE,Change,88.00000,0
2019-05-01,Service Delivery - RBO + SDP + EASE - Te,CCE,Change,84.00000,0
2019-06-01,Service Delivery - RBO + SDP + EASE - Te,CCE,Change,76.00000,0
2019-07-01,Service Delivery - RBO + SDP + EASE - Te,CCE,Change,70.00000,0
2019-08-01,Service Delivery - RBO + SDP + EASE - Te,CCE,Change,55.00000,0
2019-09-01,Service Delivery - RBO + SDP + EASE - Te,CCE,Change,58.00000,0
2019-10-01,Service Delivery - RBO + SDP + EASE - Te,CCE,Change,65.00000,0


In [6]:
# Merge our dataframes from scenario1,2,etc and remove columns with values of 0's only

final_df <- rbind(scenario1,scenario1)
 
# Keep columns that sum up greater than 0 (removes negative and columns with 0's only)
final_df <- final_df[, colSums(final_df != 0) > 0]

In [7]:
str(final_df)

'data.frame':	960 obs. of  309 variables:
 $ Date                                        : Factor w/ 48 levels "2019-01-01","2019-02-01",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ Scenario                                    : Factor w/ 1 level "Field Ops - Mass Markets - Test 7/7/2021": 1 1 1 1 1 1 1 1 1 1 ...
 $ Product                                     : Factor w/ 3 levels "EAST","NORTHERN",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Sub.Product                                 : Factor w/ 10 levels "AZ","FL_TX","GREAT PLAINS",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ Volumes                                     : int  52567 45137 50092 51663 46423 46858 56177 53765 43548 48903 ...
 $ c1_AAA_ets                                  : num  0 0 0 0 0 0 0 0 0 0 ...
 $ c1_AAN_ets                                  : num  0 0 0 0 0 0 0 0 0 0 ...
 $ c1_add_holtlineartrend                      : num  0 0 0 0 0 0 0 0 0 0 ...
 $ c1_add_holtlineartrenddamp                  : num  0 0 0 0 0 0 0 0 0 0 ...
 $ c1_ANA_ets                 

In [8]:
write.csv(final_df,"sql_export.csv", row.names = FALSE)